# Processing cellpy batch - ica

### `{{cookiecutter.project_name}}::{{cookiecutter.session_id}}`

**Experimental-id:** `{{cookiecutter.notebook_name}}`  
**Short-name:** `{{cookiecutter.session_id}}`  
**Project:** `{{cookiecutter.project_name}}`  
**By:** `{{cookiecutter.author_name}}`   
**Date:** `{{cookiecutter.date}}`  

## Imports and setup

In [ ]:
import matplotlib.pyplot as plt
import panel as pn
import holoviews as hv

import cellpy
from cellpy.utils import batch, helpers, plotutils, collectors

In [ ]:
%matplotlib inline
batch.pd.set_option('display.max_columns', 70)
print(f"cellpy version: {cellpy.__version__}")

In [ ]:
cellpy_batch_file = "cellpy_batch_{{cookiecutter.session_id}}.json"

In [ ]:
b = batch.from_journal(cellpy_batch_file)
b.link()
print(f"cell names:")
b.cell_names

In [ ]:
# # Remove bad cycles (if you have saved it to the journal session)
# helpers.yank_outliers(b, remove_indexes=b.journal.session["bad_cycles"])

In [ ]:
# # It is usually nice to see the capacity plot in the same notebook.
# b.plot_summary()

## Tweaking dQ/dV parameters

### App for tweaking the parameters

In [ ]:
XMIN = 1
XMAX = 5

pn.extension()
pn.config.console_output = "disable"

b = batch.from_journal(cellpy_batch_file)
b.link()

x_range = pn.widgets.RangeSlider(name="x_range", start=XMIN, end=XMAX, step=0.1)

# voltage step size:
voltage_res = pn.widgets.FloatInput(name="voltage_res", value=0.001, start=0.0001, end=0.1, step=0.0001, page_step_multiplier=100)

# resolution in the capacity (only valid if max_points is None (<1)):
capacity_resolution = pn.widgets.FloatInput(name="capacity_resolution", value=0.0001, start=0.0001, end=10, step=0.0001, page_step_multiplier=1000)

# max number of points for capacity (along dQ-axis):
max_points = pn.widgets.FloatInput(name="max_points", value=1000, start=0, end=10_000, step=10, page_step_multiplier=100)

# voltage resolution used for final gaussian smoothing:
voltage_fwhm = pn.widgets.FloatInput(name="voltage_fwhm", value=0.001, start=0.00001, end=0.1, step=0.0001, page_step_multiplier=100)


cycle_number = pn.widgets.IntSlider(name="cycle_number", value=1, start=1, end=2)
cell_name = pn.widgets.Select(name="cell_name", options=b.cell_names)


def cell(name):
    return b.cells[name]


def dqdv_plot(voltage, incremental, label="one", xrange=(XMIN, XMAX)):
    s1 = hv.Scatter((voltage, incremental), label=label).opts(width=800, height=600, xlabel="Voltage", ylabel="dqdv", xlim=xrange)
    s2 = hv.Curve((voltage, incremental), label=label).opts(width=800, xlabel="Voltage", ylabel="dqdv", xlim=xrange)
    return s1 * s2

def tweak_dqdv_prms(cell_name, v_res=0.01, n=2, view_fn=dqdv_plot, increment_method="diff", capacity_resolution=None, voltage_fwhm=None, max_points=None, xrange=(XMIN, XMAX)):
    cell_data = b.cells[cell_name]
    _cycles = cell_data.get_cycle_numbers()
    c_min = int(_cycles.min())
    c_max = int(_cycles.max())
    cycle_number.start = c_min
    cycle_number.end = c_max
    if cycle_number.end > c_max:
        cycle_number.value = c_max
    if max_points < 1:
        max_points = None
    cycle = cell_data.get_cap(cycle=n, categorical_column=True, method="forth-and-forth", insert_nan=False)
    if not cycle.empty:
        v, dq = ica.dqdv_cycle(cycle, voltage_resolution=v_res, increment_method=increment_method, voltage_fwhm=voltage_fwhm, capacity_resolution=capacity_resolution, max_points=max_points)
        return view_fn(v, dq, label=f"no: {n}/{c_max} - res: {v_res}", xrange=xrange)
    else:
        return "no data"


interactive = pn.bind(tweak_dqdv_prms, cell_name=cell_name, v_res=voltage_res, n=cycle_number, capacity_resolution=capacity_resolution, voltage_fwhm=voltage_fwhm, max_points=max_points, xrange=x_range)
app = pn.Row(
        pn.Column(
            cell_name,
            x_range,
            cycle_number, 
            voltage_res,
            capacity_resolution,
            max_points,
            voltage_fwhm,
        ),
        interactive
)
app

## Plotting

### By cycles

In [ ]:
cycles_collected = collectors.BatchICACollector(b, plot_type="fig_pr_cycle", plotter_arguments={"cycles": [1,2,3], "palette":"Category20"})

In [ ]:
cycles_collected.show()

In [ ]:
cycles_collected.save()

### By cells

In [ ]:
cells_collected = collectors.BatchICACollector(b)

In [ ]:
cells_collected.show()

In [ ]:
cells_collected.save()

In [ ]:
cells_collected.redraw((hv.opts.Curve(xlim=(0,2), xlabel="Voltage (V)", ylabel="dQ/dV (arb. units)"), hv.opts.NdLayout(title="All the plots")), extension="matplotlib")

## Links

### Notebooks
- notes and information [link](00_{{cookiecutter.notebook_name}}_notes.ipynb)
- processing raw data [link](01_{{cookiecutter.notebook_name}}_loader.ipynb)
- life [link](02_{{cookiecutter.notebook_name}}_life.ipynb)
- cycles [link](03_{{cookiecutter.notebook_name}}_cycles.ipynb)
- ica [link](04_{{cookiecutter.notebook_name}}_ica.ipynb)
- plots [link](05_{{cookiecutter.notebook_name}}_plots.ipynb)